Click the button below if you want to see the code behind the widgets. If you do click  and want to go back to the 'no-code' view, scroll down until you find the 'hide code' button.

In [1]:
"""
MIT License

Copyright (c) 2020 Sylvain Barde - University of Kent

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams
from IPython.display import display, HTML

javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Click to show code", True: "Click to hide code"}

def toggle_code(state):
    """
    Toggles the JavaScript show()/hide() function on the div.input element.
    """

    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)

    display(HTML(output))
    
def button_action(value):
    """
    Calls the toggle_code function and updates the button description.
    """
    state = value.new

    toggle_code(state)

    value.owner.description = button_descriptions[state]

def Elasticities_widget(Qmax_init = 15, Pmax_init = 30, Qval_init = 5, a_d_init = 2,
    b_d_init = 25, a_s_init = 2, b_s_init = -3, Dflag_init = True, Sflag_init = False):
        
    # Declare widgets for interactive input
    Qmax_slider = widgets.IntSlider(min=5,
                                 max=1000,
                                 step=1,
                                 description=r'Maximum $Q$:',
                                 value = Qmax_init,
                                 continuous_update =False)
    Pmax_slider = widgets.IntSlider(min=5,
                                 max=1000,
                                 step=1,
                                 description=r'Maximum $P$:',
                                 value = Pmax_init,
                                 continuous_update =False)
    Qval_slider = widgets.FloatSlider(min=0.001,
                                 max=15,
                                 description='Pick a Quantity:',
                                 value = Qval_init,
                                 continuous_update =False)
    a_d_slider = widgets.FloatSlider(min=0.001,
                                 max=100,
                                 description= r'$a_d$:',
                                 value = a_d_init,
                                 continuous_update =False)
    b_d_slider = widgets.FloatSlider(min=0,
                                 max=1000,
                                 description= r'$b_d$:',
                                 value=b_d_init,
                                 continuous_update =False)
    a_s_slider = widgets.FloatSlider(min=0.001,
                                 max=100,
                                 description= r'$a_s$:',
                                 value = a_s_init,
                                 continuous_update =False)
    b_s_slider = widgets.FloatSlider(min=-1000,
                                 max=1000,
                                 description= r'$b_s$:',
                                 value = b_s_init,
                                 continuous_update =False)
    Dflag_check = widgets.Checkbox(value = Dflag_init,
                                   description='Include Demand',
                                   disabled=False,
                                   indent=True) 
    Sflag_check = widgets.Checkbox(value = Sflag_init,
                                   description='Include Supply',
                                   disabled=False,
                                   indent=True)
    
    # Link widgets as required
    widgets.jslink((Qmax_slider,'value'),(Qval_slider,'max'))
    widgets.jslink((b_d_slider,'value'),(b_s_slider,'max'))

    def elasticity_plot(Qmax, Pmax, Qval, a_d, b_d, a_s, b_s, Dflag, Sflag):

        # create a quantity vector, calculate supply/demand vectors
        Q = np.arange(0,Qmax)
        P_s = a_s*Q + b_s
        P_d = -a_d*Q + b_d

        # Calculate equilibrium quantity/price
        Qeq = (b_d-b_s)/(a_s + a_d)
        Peq = a_s*Qeq + b_s

        # Calculate prices for selected value
        Pval_s = a_s*Qval + b_s
        Pval_d = -a_d*Qval + b_d

        # Create figure, plot supply/demand
        fig, ax = plt.subplots(figsize=(20,10))
        if Sflag is True:
            ax.plot(Q, P_s,'r', linewidth=2, alpha=0.6,
                    label=r'Inverse Supply $\quad P_s = a_s Q + b_s$')
        if Dflag is True:
            ax.plot(Q, P_d,'b', linewidth=2, alpha=0.6,
                    label=r'Inverse Demand $\quad P_d = -a_d Q + b_d$')

        # Add markers for the price/quantity points, with dotted lines
        mrkrSize = 2*rcParams['lines.markersize'] ** 2
        if Sflag is True and Dflag is True:
            ax.scatter(Qeq, Peq, s=mrkrSize, c='k', label='Equilibrium')       
            ax.scatter(Qval, Pval_s, s=mrkrSize, c='k', alpha=0.6, 
                       label='Selection')
            ax.scatter(Qval, Pval_d, s=mrkrSize, c='k', alpha=0.6)

            ax.plot([Qeq,Qeq],[0,Peq],'k--',linewidth=1)
            ax.plot([0,Qeq],[Peq,Peq],'k--',linewidth=1)
            ax.plot([0,Qval],[Pval_d,Pval_d],'k--',linewidth=1)
            ax.plot([Qval,Qval],[0,max(Pval_s,Pval_d)],'k--',linewidth=1)
            ax.plot([0,Qval],[Pval_s,Pval_s],'k--',linewidth=1)
            ax.annotate(r'$Q^*$',[Qeq,0], xytext = [Qeq+0.15,0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)
            ax.annotate(r'$P^*$',[0,Peq], xytext = [0.15,Peq+0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)
            ax.annotate(r'$Q$',[Qval,0], xytext = [Qval+0.15,0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)
            ax.annotate(r'$P_s$',[0,Pval_s], xytext = [0.15,Pval_s+0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)
            ax.annotate(r'$P_d$',[0,Pval_d], xytext = [0.15,Pval_d+0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)

        elif Sflag is True and Dflag is False:
            ax.scatter(Qval, Pval_s, s=mrkrSize, c='k', alpha=0.6, 
                       label='Selection')
            ax.plot([0,Qval],[Pval_s,Pval_s],'k--',linewidth=1)
            ax.plot([Qval,Qval],[0,Pval_s],'k--',linewidth=1)    
            ax.annotate(r'$Q$',[Qval,0], xytext = [Qval+0.15,0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)
            ax.annotate(r'$P_s$',[0,Pval_s], xytext = [0.15,Pval_s+0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)

        elif Sflag is False and Dflag is True:
            ax.scatter(Qval, Pval_d, s=mrkrSize, c='k', alpha=0.6, 
                       label='Selection')
            ax.plot([0,Qval],[Pval_d,Pval_d],'k--',linewidth=1)
            ax.plot([Qval,Qval],[0,Pval_d],'k--',linewidth=1)
            ax.annotate(r'$Q$',[Qval,0], xytext = [Qval+0.15,0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)
            ax.annotate(r'$P_d$',[0,Pval_d], xytext = [0.15,Pval_d+0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)

        # Add elasticity annotations
        if Sflag is True:
            EsStr = r'$E_p^S = {:.2f}$'.format(Pval_s/(Qval*a_s))
            ax.annotate(EsStr,[Qval,Pval_s],
                        xytext = [Qval+0.25,Pval_s-1], 
                        xycoords ='data',
                        fontsize = 25,
                        clip_on = True)
        if Dflag is True:
            EdStr = r'$E_p^D = {:.2f}$'.format(-Pval_d/(Qval*a_d))
            ax.annotate(EdStr,[Qval,Pval_d],
                        xytext = [Qval+0.25,Pval_d],
                        xycoords ='data',
                        fontsize = 25,
                        clip_on = True)

        # Add legend and format axes to look nice
        if Sflag is True or Dflag is True:
            ax.legend(loc='upper center', frameon=False,prop={'size':20})
        ax.autoscale(enable=True, axis='both', tight=True)
        ax.set_ylim(top = Pmax, bottom = 0)
        ax.set_xlim(right = Qmax, left = 0)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_xlabel(r'$Q$', fontdict = {'fontsize': 25},position=(1, 0))
        ax.set_ylabel(r'$P$', fontdict = {'fontsize': 25},position=(0, 1), rotation=0)
        ax.plot(1, 0, ">k", transform=ax.get_yaxis_transform(), clip_on=False)
        ax.plot(0, 1, "^k", transform=ax.get_xaxis_transform(), clip_on=False)
        plt.tick_params(labelsize=20)
        plt.tight_layout()
    
    out = widgets.interactive_output(elasticity_plot, {'Qmax': Qmax_slider,
                                                       'Pmax': Pmax_slider,
                                                       'Qval': Qval_slider, 
                                                       'a_d': a_d_slider,
                                                       'b_d': b_d_slider,
                                                       'a_s': a_s_slider, 
                                                       'b_s': b_s_slider, 
                                                       'Dflag': Dflag_check,
                                                       'Sflag': Sflag_check})

    output = widgets.VBox([out,
                  widgets.HBox([Qval_slider,
                               Qmax_slider,
                               Pmax_slider]),
                  widgets.HBox([Dflag_check, 
                                a_d_slider, 
                                b_d_slider]),
                  widgets.HBox([Sflag_check, 
                                a_s_slider, 
                                b_s_slider])])
    display(output)

def log_demand_widget(Pmax_init = 15,Qmax_init = 10, Pval1_init = 5, Pval2_init = 5, a_d_init = 0.8, b_d_init = 1.5):

    # Declare widgets for interactive input
    Qmax_slider = widgets.IntSlider(min= 5,
                                 max=50,
                                 step=1,
                                 description=r'Maximum $Q$:',
                                 value = Qmax_init,
                                 continuous_update =False)
    Pmax_slider = widgets.IntSlider(min=5,
                                 max=50,
                                 step=1,
                                 description=r'Maximum $P$:',
                                 value = Pmax_init,
                                 continuous_update =False)
    Pval1_slider = widgets.FloatSlider(min=0,
                                 max=50,
                                 description='Pick $P_1$:',
                                 value = Pval1_init,
                                 continuous_update =False)
    Pval2_slider = widgets.FloatSlider(min=0,
                                 max=15,
                                 description='Pick $P_2$:',
                                 value = Pval1_init,
                                 continuous_update =False)
    a_d_slider = widgets.FloatSlider(min=0,
                                 max=4,
                                 description= r'$\alpha$:',
                                 value = a_d_init,
                                 continuous_update =False)
    b_d_slider = widgets.FloatSlider(min=0,
                                 max=4,
                                 description= r'$\beta$:',
                                 value=b_d_init,
                                 continuous_update =False)
    
    # Link widgets as required
    widgets.jslink((Pmax_slider,'value'),(Pval1_slider,'max'))

    def log_demand_plot(Pmax, Qmax, Pval1, Pval2, a_d, b_d):

        # create a quantity vector, calculate demand/Total revenue vectors
        P = np.arange(0.01,Pmax,(Pmax-0.01)/500)
        p = np.log(P)
        pval1 = np.log(Pval1)

        Q = (P**(-a_d)) * (Pval2**b_d)
        q = -a_d*p + b_d*np.log(Pval2)

        # Calculate prices/revenues for selected value
        Qval = (Pval1**(-a_d)) * (Pval2**b_d)
        qval = -a_d*pval1 + b_d*np.log(Pval2)

        # Create figure
        mrkrSize = 2*rcParams['lines.markersize'] ** 2
        fig, ax = plt.subplots(nrows=1, ncols=2,figsize=(20,10))

        # Plot demand
        ax[0].plot(P, Q,'b', linewidth=2, alpha=0.6,
                    label=r'Demand $\quad Q_1 = P_1^{-\alpha} P_2^{\beta}$')

        # Add markers for the price/quantity points, with dotted lines
        ax[0].scatter(Pval1, Qval, s=mrkrSize, c='k', alpha=0.6,
                    label='Selection')
        ax[0].plot([0,Pval1],[Qval,Qval],'k--',linewidth=1)
        ax[0].plot([Pval1,Pval1],[0,Qval],'k--',linewidth=1)
        ax[0].annotate(r'$P_1={:.2f}$'.format(Pval1),[Qval,0], 
                    xytext = [Pval1+0.15,0.25], xycoords ='data', fontsize = 25, 
                    clip_on = True)
        ax[0].annotate(r'$Q_1={:.2f}$'.format(Qval),[0,Qval], 
                    xytext = [0.15,Qval+0.25], xycoords ='data', fontsize = 25, 
                    clip_on = True)

        # Add legend and format axes to look nice
        ax[0].legend(loc='upper center', frameon=False,prop={'size':20})
        ax[0].autoscale(enable=True, axis='both', tight=True)
        ax[0].set_ylim(top = Qmax, bottom = 0)
        ax[0].set_xlim(right = Pmax, left = 0)
        ax[0].spines['top'].set_visible(False)
        ax[0].spines['right'].set_visible(False)
        ax[0].set_xlabel(r'$P$', fontdict = {'fontsize': 25},position=(1, 0))
        ax[0].set_ylabel(r'$Q$', fontdict = {'fontsize': 25},position=(0, 1), rotation=0)
        ax[0].plot(1, 0, ">k", transform=ax[0].get_yaxis_transform(), clip_on=False)
        ax[0].plot(0, 1, "^k", transform=ax[0].get_xaxis_transform(), clip_on=False)
        ax[0].tick_params(labelsize=20)

        # Plot Logarithnmic demand
        ax[1].plot(p, q,'b', linewidth=2, alpha=0.6,
                    label=r'Log Demand $\quad q_1 = -\alpha p_1 + \beta p_2$')

        # Add markers for the price/quantity points, with dotted lines
        ax[1].scatter(pval1, qval, s=mrkrSize, c='k', alpha=0.6,
                    label='Selection')
        ax[1].plot([0,pval1],[qval,qval],'k--',linewidth=1)
        ax[1].plot([pval1,pval1],[0,qval],'k--',linewidth=1)
        ax[1].annotate(r'$\log P_1={:.2f}$'.format(pval1),[qval,0], 
                    xytext = [pval1+0.05,0.1], xycoords ='data', fontsize = 25, 
                    clip_on = True)
        ax[1].annotate(r'$\log Q_1={:.2f}$'.format(qval),[0,qval], 
                    xytext = [0.05,qval+0.1], xycoords ='data', fontsize = 25, 
                    clip_on = True)

        # Add legend and format axes to look nice
        ax[1].legend(loc='upper center', frameon=False,prop={'size':20})
        ax[1].autoscale(enable=True, axis='both', tight=True)
        ax[1].set_ylim(top = np.log(Qmax), bottom = 0)
        ax[1].set_xlim(right = np.log(Pmax), left = 0)
        ax[1].spines['top'].set_visible(False)
        ax[1].spines['right'].set_visible(False)
        ax[1].set_xlabel(r'$\log P$', fontdict = {'fontsize': 25},position=(1, 0))
        ax[1].set_ylabel(r'$\log Q$', fontdict = {'fontsize': 25},position=(0, 1), rotation=0)
        ax[1].plot(1, 0, ">k", transform=ax[1].get_yaxis_transform(), clip_on=False)
        ax[1].plot(0, 1, "^k", transform=ax[1].get_xaxis_transform(), clip_on=False)
        ax[1].tick_params(labelsize=20)

        plt.tight_layout()
    
    out = widgets.interactive_output(log_demand_plot, {'Qmax': Qmax_slider,
                                                       'Pmax': Pmax_slider,
                                                       'Pval1': Pval1_slider,
                                                       'Pval2': Pval2_slider,
                                                       'a_d': a_d_slider,
                                                       'b_d': b_d_slider})

    output = widgets.VBox([out,
                  widgets.HBox([Qmax_slider,
                               Pmax_slider,
                               Pval1_slider,
                               Pval2_slider]),
                  widgets.HBox([a_d_slider, 
                                b_d_slider])])
    
    display(output)    


In [2]:
state = False
toggle_code(state)

button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")

display(button)

ToggleButton(value=False, description='Click to show code')

### Why logarithms give economists the best of both worlds

This notebook aims to show you why economists are so fond of logarithms.

#### The elephant in the room: Linear 'curves'

Below is the market diagram that was used in the very first notebook, which aimed at explaining the concept of elasticity. Like many of the demand/supply diagrams we have seen in other notebooks, or like many of the diagrams that you will see in economics textbooks, it shows a **linear** demand curve.

From a mathematical point of view, that's a good thing: linear functions are easy to work with, their slope is constant, and if you have several linear equations (i.e. a system of equations), they are relatively easy to solve, either with by hand using substitution methods or in a computer, using the more powerful matrix approach.


In [3]:
Elasticities_widget()

But from an *economic* point of view we have a massive elephant in our room: a linear function must intersect the axes. We clearly have a point where $P=0$, and the curve tells us that at that point, the quantity demanded is a reasonable 12.5 units. Similarly we have a point where the quantity demanded is 0 units, which happens when the price is 25. If you stop and think for a second, this doesn't match with our everyday experience of the real-life economy:
- When a good becomes extremely scarce (the quantity available on the market tends to 0), the price people are willing to pay will become disproportionally higher the lower the quantity. This can reach a point where extremely scarce goods (maybe 1-2 units available) have to be sold at auction so that the good goes to the people who value it the most. A good examples here are paintings by Vincent van Gogh... 
- On the other hand, when a good is so cheap as to become essentially free, then demand for it can expand disproportionally. A 'reverse' example here are plastic carrier bags in supermarkets, which where initially free of charge. Because they were perceived as having no value, they were overused and thrown away carelessly, with all the environmental damage we know about. A simple increase in price from 0p (free) to 5p per bag [reduced the quantity demanded by consumers by 90%](https://www.independent.co.uk/news/uk/politics/plastic-bag-charge-supermarkets-figures-reduction-a9029996.html)

The solution to this problem is simple: use a non-linear demand function, which results in a true **curve**. This way, you can capture these two effects: quantity demanded will expand very fast as price goes to zero, and price will rise very quickly when the quantity people can buy falls to zero. It is often the case that reality requires us to use ***non-linear*** functions!

But wouldn't it be nice if we could make our functions non-linear, so that they are more realistic, but keep the nice properties of linear functions?

#### Logarithms to the rescue...

Life doesn't often allow us to have our cake and eat it, but this is one of the rare occasions where we can. Consider the following non-linear demand function for good 1, where the quantity demanded for good 1, $Q_1$, is a function of the price of good 1, $P_1$, and the price of a second good, $P_2$. $\alpha$ and $\beta$ are two positive parameters, and they make the quantity demanded of good 1 a negative function of $P_1$ (as expected) and a positive function of the price of the other good $P_2$ (in other words, the goods are substitutes, like tea and coffee).

$$Q_1 = P_1^{-\alpha}P_2^{\beta}$$

This function is plotted on the left hand side plot in the diagram below. Note that this plots quantity as a function of price, unlike the plot above, which shows the inverse demand curve where price is a function of quantity. If you look at the plot, you will see that this demand curve is more realistic. For very low values of the price, the quantity demanded shoots up (think free plastic carrier bags) and at very high prices the quantity demanded becomes low, but doesn't completely vanish (think rare items in auctions).

In addition to this, the specific equation picked for the demand function has a very interesting property: it becomes **linear** if you take its logarithm! Let's see this step by step. First, we take the natural log of both sides of the equation.

$$\ln Q_1 = \ln \left(P_1^{-\alpha}P_2^{\beta}\right)$$

On the left hand side, not much will happen. On the right hand side, because $\ln(xy) = \ln(x) +\ln(y)$ we can split the product of the terms containing $P_1$ and $P_1$. 

$$\ln Q_1 = \ln \left(P_1^{-\alpha}\right)  + \ln \left(P_2^{\beta}\right)$$

Next, because $\ln(x^n) = n\ln(x)$, we can bring the $\alpha$ and $\beta$ exponents down in front of $\ln \left(P_1\right)$ and $\ln \left(P_2\right)$.

$$\ln Q_1 = -\alpha \ln P_1 + \beta \ln P_2$$

Finally, for convenience, we substitute $q_1 = \ln Q_1$, $p_1 = \ln P_1$ and $p_2 = \ln P_2$. In economics, it is very common for the level of a variable to be in uppercase, and the logarithm of that variable to be in lowercase.

$$q_1 = -\alpha p_1 + \beta p_2$$

Hey presto, by taking the logarithm of the non-linear (curved) demand function we have recovered a linear function! This is the right-hand-side plot below.

In [4]:
log_demand_widget()

Functions like the curved demand function shown above are sometimes known as 'log linear', as they become linear when you take logarithms of the variables. This gives us the best of both worlds: are functions can be non-linear when we need them to describe reality and linear when we need to work with them.

#### A practical illustration

Let's take a quick look at how useful this can be with a system of equations. Suppose we have two demand equations like the one above, for two substitute goods like tea and coffee. Each of these reacts negatively to its own price, and positively to the other price:

$$\left\{\begin{aligned}
Q_1 & = P_1^{-\alpha}P_2^{\beta} \\
Q_2 & = P_1^{\gamma}P_2^{-\delta} \\
\end{aligned}\right.$$

Imagine you know the parameters of these demands ($\alpha$, $\beta$,$\gamma$,$\delta$) you know  the quantities of tea and coffee that are available on both markets ($Q_1$ and $Q_2$), and you want to find out the prices that will clear these two markets. You have two unknown variables ($P_1$ and $P_2$), and two equations to solve them. All you need to do is solve the system! Solving the system above is absolutely possible, it can be done by substitution: you isolate one price (say $P_1$) in the first variable, substitute in the second. I am sure that you can see, however, that this requires working with powers in a way that adds quite a bit more complication.

Instead you can  make your life easier by using the logarithmic form of the system of equations. Notice how much easier this problem looks now:

$$\left\{\begin{aligned}
q_1 & = -\alpha p_1 + \beta p_2\\
q_2 & =  \gamma p_1 - \delta p_2\\
\end{aligned}\right.$$

All we need to do now is use the methods we saw in the system of equations videos, either substitution or direct elimination. If you have a larger demand system, with more goods and more prices, you can see that you could also write this as a matrix, feed it to a computer and get the solutions for the log prices $p_i$. Once you have your log prices $p_1$, $p_2$, etc. the only extra step you need to do is take the exponential to recover the prices $P_1 = e^{p_1}$, $P_2 = e^{p_1}$.

I'm not going to work through this system in detail (I've not actually specified any numbers here!), that's not the point of the notebook. Instead, the key idea I wanted to show you is how much easier to solve the second system looks compared to the first!

#### Wrapping up...

This little example was designed to serve two purposes. The first is to give you an illustration of why logarithms are so useful in economics: they allow us to account for some of the features of real-life economies at very little 'mathematical. cost. As long as we pick the right type of function, we can have non-linear features but still work in linear terms if we want too. Next year, when you take econometrics, you will see that while the basic tools we use typically allow us to estimate **linear** functions, using logs allows us to get around this to fit non-linear data.

The second purpose is to give you an understanding of why economists often use functions that look like that demand curve: we use these curves as they are a good compromise between being realistic and having a tool that is relatively easy to work with. That doesn't mean that we 100% believe that the world follows such equations **exactly**, just that we think that they are close enough while still being usable. We will see examples of these later in the term, when we start looking at multivariate calculus.